<h2>Twitter Sentiment Analysis With Scikit-Learn</h2>
<p>In this lesson we will use the scikit-learn Python library to perform some natural language processing tasks on our Twitter #climatechange dataset to evaluate user sentiment within the  280 character chunk of tweet text. For a supervised learning task, one must specify training and test datasets. For this example, we will use Textblob's built-in sentiment analysis to generate the core of our training and test datasets. </p>
<p>This process consists of running the Textblob polarity sentiment analysis on the tweet text from our #climatechange dataset and extract a subset of the most positive or negative tweet text and create two datasets large enough to give us reasonable results but small enough to process quickly. </p>
<p>We needed a reasonably large sample size so we chose weets for the positive dataset which had polarity sentiment values greater than 0.6 and tweets in the negative dataset which had polarity sentiment less than 0.4. This resulted in ???? positive tweets and ???? negative tweets. </p>
<p>We then filtered the data to remove end-of-line characters, special characters, multiple sequential spaces, single character words, etc. 


<h2>Import Packages</h2>

In [ ]:
import re
#import datetime
#import os
#import sys
#import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from textblob import TextBlob
#import sklearn
   

## Import #climatechange Dataset

In [ ]:
data_dir = '/data/elastic-notebook/data/cvw/twitter/'

In [ ]:
df = pd.read_csv(data_dir + 'climatechange_tweets_all.csv')

In [ ]:
df.info()

## Create a Filtered Training/Test Dataset of Polarity Sentiment Measure Using Textblob

In order to carry out classification, one needs to have labeled data examples: a set of inputs, along with associated labels, so that a classification algorithm can learn how to disciminate among the different labels based on the input.  Creating such a dataset (sometimes referred to as a "gold standard" dataset) can be very time consuming.

In this exercise here, we have a set of tweets, and we'd like to 

<p>Creating a training/test dataset can require a significant amount of time. Often crowd-sourcing is used for this purpose. Since this is just a tutorial based on 'low-stakes' examples, we will use a simpler and quicker method to create our training/test dataset with Textblob. Textblob provides a feature designed to perform sentiment analysis for two different measures of sentiment, polarity and subjectivity. In this example we will use polarity, i.e. the negative or positive nature of a text sample, to classify our #climatechange dataset into mostly positive and mostly negative datasets. </p>

<p>In the next code cell, we filter the data in several ways before processing with TextBlob. </p>

In [ ]:
dff = df.drop_duplicates(subset='retweet_id')[['retweet_id', 'text']]
count = 0
for idx, row in dff.iterrows():
    # Remove end-of-line characters
    tweet_text = re.sub('\n','',row['text'])
    # Remove special characters
    tweet_text = re.sub(r'\W', ' ', tweet_text)
    # remove all single characters
    tweet_text = re.sub(r'\s+[a-zA-Z]\s+', ' ', tweet_text)
    # Remove single characters from the start
    tweet_text = re.sub(r'\^[a-zA-Z]\s+', ' ', tweet_text) 
    # Substituting multiple spaces with single space
    tweet_text = re.sub(r'\s+', ' ', tweet_text, flags=re.I)
    # Removing prefixed 'b'
    tweet_text = re.sub(r'^b\s+', '', tweet_text)
    # Converting to Lowercase
    tweet_text = tweet_text.lower()
    # Calculate the Sentiment with TextBlob
    tweet_text_sentiment = TextBlob(tweet_text)
    #
    dff.loc[idx, 'tweet_text'] = tweet_text
    dff.loc[idx, 'polarity'] = tweet_text_sentiment.sentiment.polarity
    dff.loc[idx, 'subjectivity'] = tweet_text_sentiment.sentiment.subjectivity
    #
    count += 1
    if count%10000 == 0:
        print(count)

dff.drop('text', axis=1, inplace=True)

In [ ]:
print(dff.info())
dff.head()

In [ ]:
dff['polarity'].plot.hist()

In [ ]:
dff['polarity_label'] = dff['polarity'].apply(
    lambda x: 'positive' if x > 0.5 else ('negative' if x < -0.3 else 'neutral'))

In [ ]:
dff['subjectivity'].plot.hist(bins=20)

In [ ]:
dff['subjectivity_label'] = dff['subjectivity'].apply(
    lambda x: 'subjective' if x > 0.9 else ('objective' if x < 0.02 else 'neither'))

In [ ]:
dff.head()

## Building a Classifier

Now that we have created our datasets we can proceed to create a classifier, the aim of which is to take as input a given tweet text and to assign a label as to its likely sentiment (positive, negative, or neutral).

In order 

<p>Next, we need to initialize a vectorizer. This converts the tweet text into a word 'vector' in which each word becomes a 'component' of the vector and has a specific magnitude. We have not performed any preprocessing on the data or removed any stop words. We will set lowercase to False.</p>

which should be able to identify with reasonable accuracy whether a tweet has a positive or negative sentiment. First, we will import our data. Note the print statement in the cell which processes the negative dataset. It is there just to give us a peek into what Textblob determined were negative tweet text.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer     

vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = False,
    max_features = 85
)

In [ ]:
features = vectorizer.fit_transform(dff.tweet_text).toarray()
features.shape

As a final step, we'll split the training data to get an evaluation set through scikit-learn's built-in cross validation method. All we need to do is provide the data and assign a training percentage (in this case, 80%).

In [ ]:
from sklearn.model_selection import train_test_split

X = features
y = dff.polarity_label

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1234)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### Build a Linear Classifier

Finally, having preprocessed the data, we can build a classifier for this corpus! As mentioned before, we'll be using the Logistic Regression from scikit-learn, so we'll start there: 

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(solver='liblinear')

Once the model is initialized, we have to fit it to our specific dataset, so we use scikit-learn's `fit()` method to do so:

In [ ]:
log_model = log_model.fit(X=X_train, y=y_train)

And finally, we use this classifier to label the evaluation set we created earlier:

In [ ]:
y_pred = log_model.predict(X_test)

### Evaluate the Model's Accuracy

In scikit-learn, there is a function called sklearn.metrics.accuracy_score which calculates the accuracy percentage. Using this, we see that this model has an accuracy of about 70%. 

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

When we ran this on our raw unfiltered data we get ~70%. That's not bad, but 76% is much better, and we could probably do even better if we manually reviewed the datasets for any misclassifications by TextBlob. 

In [ ]:
from sklearn.model_selection import cross_validate
cross_validate(log_model, X, y, cv=5)

Let's try some other classifiers, such as Naive Bayes. Scikit-learn offers a few variations on the Naive-Bayes classifier, Gaussian, Multinomial, and Complement. We will create and test each model and compare our results with the Logistic Regression model. 

<h2>Gaussian NB Classifier</h2>
<p>This classifier does not really make sense because it assumes the distribution of our data is gaussian, which is not likely for tweet text. But let's run it anyhow and confirm our suspicions.</p>

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb_model = GaussianNB()
cross_validate(gnb_model, X, y, cv=5)
#gnb_model = gnb.fit(X=X_train, y=y_train)
#y_pred = gnb_model.predict(X_test)

In [ ]:
#from sklearn.metrics import accuracy_score
#print(accuracy_score(y_test, y_pred))

In [ ]:
#len(y_train[y_train == 'negative'])

<h2>Multinomial NB Classifier</h2>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb_model = MultinomialNB()
cross_validate(mnb_model, X, y, cv=5)
#mnb_model = mnb.fit(X=X_train, y=y_train)
#y_pred = mnb_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

<h2>Complement NB Classifier</h2>

In [ ]:
from sklearn.naive_bayes import ComplementNB
cnb_model = ComplementNB()
cross_validate(cnb_model, X, y, cv=5)
#cnb_model = cnb.fit(X=X_train, y=y_train)
#y_pred = cnb_model.predict(X_test)

It appears none of the Naive Bayes classifiers performs better than Logistic Regression. 

In [ ]:
retweet_ids = set()
retweet_id_list = []
positive_tweets = []
negative_tweets = []
polarity = []
subjectivity = []
tweet_text_list = []
val_inc = 0

for index, row in df.iterrows():
    # Check if retweet id is in set and process if not
    if row['retweet_id'] not in retweet_ids:
        retweet_ids.add(row['retweet_id'])
        retweet_id_list.append(row['retweet_id'])
        # Remove end-of-line characters
        tweet_text = re.sub('\n','',row['text'])
        # Remove special characters
        tweet_text = re.sub(r'\W', ' ', tweet_text)
        # remove all single characters
        tweet_text = re.sub(r'\s+[a-zA-Z]\s+', ' ', tweet_text)
        # Remove single characters from the start
        tweet_text = re.sub(r'\^[a-zA-Z]\s+', ' ', tweet_text) 
        # Substituting multiple spaces with single space
        tweet_text = re.sub(r'\s+', ' ', tweet_text, flags=re.I)
        # Removing prefixed 'b'
        tweet_text = re.sub(r'^b\s+', '', tweet_text)
        # Converting to Lowercase
        tweet_text = tweet_text.lower()
        # Calculate the Sentiment with TextBlob
        tweet_text_sentiment = TextBlob(tweet_text)
        polarity.append(tweet_text_sentiment.sentiment.polarity)
        subjectivity.append(tweet_text_sentiment.sentiment.subjectivity)
        tweet_text_list.append(tweet_text)
        val_inc += 1
        if val_inc%10000 == 0:
            print(str(val_inc))


In [ ]:
polarity_df = pd.DataFrame(retweet_id_list, columns=['retweet_id'])

In [ ]:
polarity_df.info()

We need to add the tweet text and polarity to our dataframe as an additional column. Since these are lists, we can trust that they retain a specific immutable sequence and thus will 'line-up' corresponding values correctly. 

In [ ]:
polarity_df['tweet_text'] = tweet_text_list

In [ ]:
polarity_df['polarity'] = polarity

In [ ]:
polarity_df.head()

# Optional Exercise: Subjectivity Data

<h2>Save Filtered Polarity Data</h2>

Let's save our results in case we need this data for future examples. 

In [ ]:
#REMOVE

polarity_df.to_csv(data_dir + 'polarity_df_clean.csv')

<h2>[Optional as an Exercise] Save Filtered Subjectivity Data</h2>

In [ ]:
#REMOVE

subjectivity_df = pd.DataFrame(retweet_id_list, columns=['retweet_id'])
subjectivity_df['tweet_text'] = tweet_text_list
subjectivity_df['subjectivity'] = subjectivity
subjectivity_df.to_csv(data_dir + 'subjectivity_df_clean.csv')

In [ ]:
subjectivity_df.head()

<h2>Extract Positive and Negative Text</h2>

In [ ]:
# REMOVE

positive_tweets_df = polarity_df[polarity_df.polarity > 0.5]
negative_tweets_df = polarity_df[polarity_df.polarity < -0.3]
positive_text = positive_tweets_df.tweet_text
negative_text = negative_tweets_df.tweet_text

<h2>[Optional as an Exercise] Extract Subjective and Objective Text</h2>

In [ ]:
subjective_tweets_df = polarity_df[subjectivity_df.subjectivity > 0.9]
objective_tweets_df = polarity_df[subjectivity_df.subjectivity < 0.02]
subjective_text = subjective_tweets_df.tweet_text
objective_text = objective_tweets_df.tweet_text

<h2>Save Filtered Polarity Data</h2>

Let's save our results in case we need this data for future examples. 

In [ ]:
positive_text.to_csv(data_dir + 'positive_text_clean.csv', index=False, header=False)
negative_text.to_csv(data_dir + 'negative_text_clean.csv', index=False, header=False)

<h2>[Optional as an Exercise] Save Filtered Subjectivity Data</h2>

In [ ]:
subjective_text.to_csv(data_dir + 'subjective_text_clean.csv', index=False, header=False)
objective_text.to_csv(data_dir + 'objective_text_clean.csv', index=False, header=False)

In [ ]:
#REMOVE

text_pos = []
labels_pos = []
with open(data_dir + 'positive_text_clean.csv', encoding='utf-8') as f:
    for i in f: 
        text_pos.append(i) 
        labels_pos.append('pos')


In [ ]:
#REMOVE


text_neg = []
labels_neg = []
with open(data_dir + 'negative_text_clean.csv', encoding='utf-8') as f:
    for i in f: 
        #print(i)
        text_neg.append(i)
        labels_neg.append('neg')

Next, we split each dataset into a training and a test dataset. 

In [ ]:
training_text = text_pos[:int((.8)*len(text_pos))] + text_neg[:int((.8)*len(text_neg))]
training_labels = labels_pos[:int((.8)*len(labels_pos))] + labels_neg[:int((.8)*len(labels_neg))]

test_text = text_pos[int((.8)*len(text_pos)):] + text_neg[int((.8)*len(text_neg)):]
test_labels = labels_pos[int((.8)*len(labels_pos)):] + labels_neg[int((.8)*len(labels_neg)):]

In [ ]:
test_frac = 0.8
breakpoint = int(test_frac * len(dff))
training = dff.iloc[:breakpoint]
testing = dff.iloc[breakpoint:]

<p>Next, we need to initialize a vectorizer. This converts the tweet text into a word 'vector' in which each word becomes a 'component' of the vector and has a specific magnitude. We have not performed any preprocessing on the data or removed any stop words. We will set lowercase to False.</p>

In [ ]:
features = vectorizer.fit_transform(
    training_text + test_text)

features_nd = features.toarray() # for easy use

In [ ]:
#REMOVE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd[0:len(training_text)], 
        training_labels, 
        random_state=124)

<p>In this next step we concatenate our text into a single array for feature extraction.</p>